In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,0,0,0,NaN,NaN,0,5,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,0,0,0,NaN,NaN,0,0,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,0,0,0,NaN,NaN,0,1,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,0,0,0,NaN,NaN,0,10,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,18,0,0,NaN,NaN,18,148,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5560,2020-11-14T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1462,215,1677,26129,...,1520,1729,11444,860,24924.0,15186.0,40110,809193,560736.0,NaN
5561,2020-11-14T17:00:00,ITA,9,Toscana,43.769231,11.255889,1679,266,1945,49116,...,932,2420,23854,1835,61678.0,15072.0,76750,1313187,848164.0,NaN
5562,2020-11-14T17:00:00,ITA,10,Umbria,43.106758,12.388247,361,68,429,10347,...,386,579,7201,242,5077.0,13142.0,18219,356390,201299.0,NaN
5563,2020-11-14T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,146,13,159,2140,...,-64,245,2564,240,4513.0,590.0,5103,49650,29478.0,NaN


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5560,2020-11-14T17:00:00,19,Sicilia,1462,215,1677,26129,27806,1520,1729,11444,860,24924.0,15186.0,40110,809193,560736.0
5561,2020-11-14T17:00:00,9,Toscana,1679,266,1945,49116,51061,932,2420,23854,1835,61678.0,15072.0,76750,1313187,848164.0
5562,2020-11-14T17:00:00,10,Umbria,361,68,429,10347,10776,386,579,7201,242,5077.0,13142.0,18219,356390,201299.0
5563,2020-11-14T17:00:00,2,Valle d'Aosta,146,13,159,2140,2299,-64,245,2564,240,4513.0,590.0,5103,49650,29478.0


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2020-11-14', '2020-11-13')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-14T17:00:00,1,Piemonte,4902,360,5262,63953,69215,2561,4471,47567,5001,50978.0,70805.0,121783,1259309,798081.0
1,2020-11-14T17:00:00,2,Valle d'Aosta,146,13,159,2140,2299,-64,245,2564,240,4513.0,590.0,5103,49650,29478.0
2,2020-11-14T17:00:00,3,Lombardia,7621,817,8438,147841,156279,5010,8129,137255,19186,236852.0,75868.0,312720,3511792,2135880.0
3,2020-11-14T17:00:00,5,Veneto,1869,228,2097,57608,59705,2417,3578,35563,2816,27166.0,70918.0,98084,2547409,984607.0
4,2020-11-14T17:00:00,6,Friuli Venezia Giulia,378,45,423,9106,9529,378,558,8284,516,15816.0,2513.0,18329,615669,252095.0
5,2020-11-14T17:00:00,7,Liguria,1402,108,1510,13824,15334,449,1091,25133,2088,29720.0,12835.0,42555,522497,264030.0
6,2020-11-14T17:00:00,8,Emilia-Romagna,2243,231,2474,48088,50562,2406,2637,30129,5008,56229.0,29470.0,85699,1846431,965000.0
7,2020-11-14T17:00:00,9,Toscana,1679,266,1945,49116,51061,932,2420,23854,1835,61678.0,15072.0,76750,1313187,848164.0
8,2020-11-14T17:00:00,10,Umbria,361,68,429,10347,10776,386,579,7201,242,5077.0,13142.0,18219,356390,201299.0
9,2020-11-14T17:00:00,11,Marche,484,74,558,12861,13419,585,779,8088,1102,22500.0,109.0,22609,365075,214152.0


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-13T17:00:00,1,Piemonte,4890,353,5243,61411,66654,3456,5258,45694,4964,49229.0,68083.0,117312,1239594,785558.0
1,2020-11-13T17:00:00,2,Valle d'Aosta,149,15,164,2199,2363,25,103,2261,234,4295.0,563.0,4858,48853,29189.0
2,2020-11-13T17:00:00,3,Lombardia,7319,801,8120,143149,151269,8476,10634,134294,19028,231380.0,73211.0,304591,3476242,2115026.0
3,2020-11-13T17:00:00,5,Veneto,1831,219,2050,55238,57288,2533,3605,34454,2764,26878.0,67628.0,94506,2529005,978328.0
4,2020-11-13T17:00:00,6,Friuli Venezia Giulia,366,47,413,8738,9151,504,831,8114,506,15378.0,2393.0,17771,610398,250805.0
5,2020-11-13T17:00:00,7,Liguria,1372,110,1482,13403,14885,224,1209,24521,2058,28704.0,12760.0,41464,516330,261534.0
6,2020-11-13T17:00:00,8,Emilia-Romagna,2164,225,2389,45767,48156,2132,2384,29947,4965,54812.0,28256.0,83068,1824983,956466.0
7,2020-11-13T17:00:00,9,Toscana,1656,265,1921,48208,50129,1213,2478,22408,1793,59899.0,14431.0,74330,1294003,839613.0
8,2020-11-13T17:00:00,10,Umbria,359,68,427,9963,10390,-394,604,7016,234,4953.0,12687.0,17640,351691,199394.0
9,2020-11-13T17:00:00,11,Marche,559,74,633,12201,12834,622,740,7902,1094,21500.0,330.0,21830,360800,211777.0


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-14T17:00:00,1,Piemonte,4902,360,5262,63953,69215,2561,4471,47567,5001,50978.0,70805.0,121783,1259309,798081.0
1,2020-11-14T17:00:00,2,Valle d'Aosta,146,13,159,2140,2299,-64,245,2564,240,4513.0,590.0,5103,49650,29478.0
2,2020-11-14T17:00:00,3,Lombardia,7621,817,8438,147841,156279,5010,8129,137255,19186,236852.0,75868.0,312720,3511792,2135880.0
3,2020-11-14T17:00:00,4,Trentino-Alto Adige,811,75,886,12007,12893,259,896,15186,895,23280.0,5694.0,28974,600205,NaN
4,2020-11-14T17:00:00,5,Veneto,1869,228,2097,57608,59705,2417,3578,35563,2816,27166.0,70918.0,98084,2547409,984607.0
5,2020-11-14T17:00:00,6,Friuli Venezia Giulia,378,45,423,9106,9529,378,558,8284,516,15816.0,2513.0,18329,615669,252095.0
6,2020-11-14T17:00:00,7,Liguria,1402,108,1510,13824,15334,449,1091,25133,2088,29720.0,12835.0,42555,522497,264030.0
7,2020-11-14T17:00:00,8,Emilia-Romagna,2243,231,2474,48088,50562,2406,2637,30129,5008,56229.0,29470.0,85699,1846431,965000.0
8,2020-11-14T17:00:00,9,Toscana,1679,266,1945,49116,51061,932,2420,23854,1835,61678.0,15072.0,76750,1313187,848164.0
9,2020-11-14T17:00:00,10,Umbria,361,68,429,10347,10776,386,579,7201,242,5077.0,13142.0,18219,356390,201299.0


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,0,2020-11-13T17:00:00,1,Piemonte,4890,353,5243,61411,66654,3456,5258,45694,4964,49229.0,68083.0,117312,1239594,785558.0
1,1,2020-11-13T17:00:00,2,Valle d'Aosta,149,15,164,2199,2363,25,103,2261,234,4295.0,563.0,4858,48853,29189.0
2,2,2020-11-13T17:00:00,3,Lombardia,7319,801,8120,143149,151269,8476,10634,134294,19028,231380.0,73211.0,304591,3476242,2115026.0
3,19,2020-11-13T17:00:00,4,Trentino-Alto Adige,806,71,877,11757,12634,566,1022,14565,879,22530.0,5548.0,28078,593328,NaN
4,3,2020-11-13T17:00:00,5,Veneto,1831,219,2050,55238,57288,2533,3605,34454,2764,26878.0,67628.0,94506,2529005,978328.0
5,4,2020-11-13T17:00:00,6,Friuli Venezia Giulia,366,47,413,8738,9151,504,831,8114,506,15378.0,2393.0,17771,610398,250805.0
6,5,2020-11-13T17:00:00,7,Liguria,1372,110,1482,13403,14885,224,1209,24521,2058,28704.0,12760.0,41464,516330,261534.0
7,6,2020-11-13T17:00:00,8,Emilia-Romagna,2164,225,2389,45767,48156,2132,2384,29947,4965,54812.0,28256.0,83068,1824983,956466.0
8,7,2020-11-13T17:00:00,9,Toscana,1656,265,1921,48208,50129,1213,2478,22408,1793,59899.0,14431.0,74330,1294003,839613.0
9,8,2020-11-13T17:00:00,10,Umbria,359,68,427,9963,10390,-394,604,7016,234,4953.0,12687.0,17640,351691,199394.0


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione
0,2020-11-14T17:00:00,1,Piemonte,4902,360,5262,63953,69215,2561,4471,47567,5001,50978.0,70805.0,121783,1259309,798081.0,4341375
1,2020-11-14T17:00:00,2,Valle d'Aosta,146,13,159,2140,2299,-64,245,2564,240,4513.0,590.0,5103,49650,29478.0,125501
2,2020-11-14T17:00:00,3,Lombardia,7621,817,8438,147841,156279,5010,8129,137255,19186,236852.0,75868.0,312720,3511792,2135880.0,10103969
3,2020-11-14T17:00:00,4,Trentino-Alto Adige,811,75,886,12007,12893,259,896,15186,895,23280.0,5694.0,28974,600205,NaN,1074819
4,2020-11-14T17:00:00,5,Veneto,1869,228,2097,57608,59705,2417,3578,35563,2816,27166.0,70918.0,98084,2547409,984607.0,4907704
5,2020-11-14T17:00:00,6,Friuli Venezia Giulia,378,45,423,9106,9529,378,558,8284,516,15816.0,2513.0,18329,615669,252095.0,1211357
6,2020-11-14T17:00:00,7,Liguria,1402,108,1510,13824,15334,449,1091,25133,2088,29720.0,12835.0,42555,522497,264030.0,1543127
7,2020-11-14T17:00:00,8,Emilia-Romagna,2243,231,2474,48088,50562,2406,2637,30129,5008,56229.0,29470.0,85699,1846431,965000.0,4467118
8,2020-11-14T17:00:00,9,Toscana,1679,266,1945,49116,51061,932,2420,23854,1835,61678.0,15072.0,76750,1313187,848164.0,3722729
9,2020-11-14T17:00:00,10,Umbria,361,68,429,10347,10776,386,579,7201,242,5077.0,13142.0,18219,356390,201299.0,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione,totale_posti_terapia_intensiva
0,2020-11-14T17:00:00,1,Piemonte,4902,360,5262,63953,69215,2561,4471,47567,5001,50978.0,70805.0,121783,1259309,798081.0,4341375,367
1,2020-11-14T17:00:00,2,Valle d'Aosta,146,13,159,2140,2299,-64,245,2564,240,4513.0,590.0,5103,49650,29478.0,125501,20
2,2020-11-14T17:00:00,3,Lombardia,7621,817,8438,147841,156279,5010,8129,137255,19186,236852.0,75868.0,312720,3511792,2135880.0,10103969,983
3,2020-11-14T17:00:00,4,Trentino-Alto Adige,811,75,886,12007,12893,259,896,15186,895,23280.0,5694.0,28974,600205,NaN,1074819,106
4,2020-11-14T17:00:00,5,Veneto,1869,228,2097,57608,59705,2417,3578,35563,2816,27166.0,70918.0,98084,2547409,984607.0,4907704,825
5,2020-11-14T17:00:00,6,Friuli Venezia Giulia,378,45,423,9106,9529,378,558,8284,516,15816.0,2513.0,18329,615669,252095.0,1211357,175
6,2020-11-14T17:00:00,7,Liguria,1402,108,1510,13824,15334,449,1091,25133,2088,29720.0,12835.0,42555,522497,264030.0,1543127,209
7,2020-11-14T17:00:00,8,Emilia-Romagna,2243,231,2474,48088,50562,2406,2637,30129,5008,56229.0,29470.0,85699,1846431,965000.0,4467118,516
8,2020-11-14T17:00:00,9,Toscana,1679,266,1945,49116,51061,932,2420,23854,1835,61678.0,15072.0,76750,1313187,848164.0,3722729,415
9,2020-11-14T17:00:00,10,Umbria,361,68,429,10347,10776,386,579,7201,242,5077.0,13142.0,18219,356390,201299.0,880285,70


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-11-14T17:00:00,3,Lombardia,7621,817,8438,147841,156279,5010,8129,...,83.11,4692,-2505,35550,20854.0,158,2961,22.87,0.08045,3.10
1,2020-11-14T17:00:00,1,Piemonte,4902,360,5262,63953,69215,2561,4471,...,98.09,2542,-787,19715,12523.0,37,1873,22.68,0.10299,2.81
2,2020-11-14T17:00:00,5,Veneto,1869,228,2097,57608,59705,2417,3578,...,27.64,2370,-27,18404,6279.0,52,1109,19.44,0.07291,2.00
3,2020-11-14T17:00:00,15,Campania,2080,184,2264,82982,85246,2026,3351,...,43.09,2098,-728,20662,17496.0,27,1298,16.22,0.05792,1.86
4,2020-11-14T17:00:00,12,Lazio,3076,273,3349,61609,64958,2559,2997,...,36.55,2488,72,29822,24696.0,39,399,10.05,0.05110,1.39
5,2020-11-14T17:00:00,8,Emilia-Romagna,2243,231,2474,48088,50562,2406,2637,...,44.77,2321,253,21448,8534.0,43,182,12.29,0.05903,1.92
6,2020-11-14T17:00:00,9,Toscana,1679,266,1945,49116,51061,932,2420,...,64.10,908,-58,19184,8551.0,42,1446,12.61,0.06501,2.06
7,2020-11-14T17:00:00,16,Puglia,1289,178,1467,22533,24000,1489,1741,...,48.63,1429,391,9745,7020.0,16,236,17.87,0.04343,0.84
8,2020-11-14T17:00:00,19,Sicilia,1462,215,1677,26129,27806,1520,1729,...,39.96,1503,22,9274,6179.0,23,186,18.64,0.03480,0.81
9,2020-11-14T17:00:00,7,Liguria,1402,108,1510,13824,15334,449,1091,...,51.67,421,-118,6167,2496.0,30,612,17.69,0.07070,2.76


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Piemonte,360,7,367,98.09
1,Umbria,68,2,70,97.14
2,Lombardia,817,166,983,83.11
3,Trentino-Alto Adige,75,31,106,70.75
4,Valle d'Aosta,13,7,20,65.00
5,Toscana,266,149,415,64.10
6,Marche,74,53,127,58.27
7,Liguria,108,101,209,51.67
8,Puglia,178,188,366,48.63
9,Emilia-Romagna,231,285,516,44.77


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['nuovi_positivi'] / (df['totale_positivi'] - df['nuovi_positivi'] + df['nuovi_dimessi_guariti'] + df['nuovi_deceduti'])).replace(np.inf, 0) * 100).round(3)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv
